# Tarea y ejercicio de clase: _Consultas SQL_ :

## Objetivos:
    1. Conectarnos a una base de datos desde Python.
    2. Enviar un query a la base de datos y exportarlo como un dataframe de pandas.
    3. Introducción a joins entre tablas.
        3.1 Teoría de Conjuntos
        3.2 Joins.
    4. Consideraciones: Subqueries. 
    
 
 
 ### 1. Conectarnos a una base de datos desde Python. 
     
En las sesiones anteriores, emulamos la conexión a una base de datos relacional importando un csv. En esta ocasión, utilizaremos el datasest _pseudo-real_ de employees. Este dataset se ha utilizado para pruebas de _benchmarking_ tanto en análisis como en desempeño.  Nosotros lo utilizaremos para establecer una conexión remota y para introducir algunas consideraciones sobre cómo hacer que nuestros queries sean más eficientes.

Pueden encontrar más información del mismo acá: 

https://relational.fit.cvut.cz/dataset/Employee
https://www.percona.com/blog/2011/02/01/sample-datasets-for-benchmarking-and-testing/

Para realizar esta conexión, utilizaremos la librería *mysql.connector*, la cual pueden instalar en el ambiente del curso por medio de Anaconda. 



In [5]:
import pandas as pd

Creamos una función la cual tendrá como parámetros lo necesario para poder establecer la conexión al SQL:

In [12]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [96]:
host = "relational.fit.cvut.cz"
user = "guest"
pwd = "relational"
connection = create_connection(host_name = host, user_name =user ,user_password = pwd)

Connection to MySQL DB successful


### 2. Enviar un query a la base de datos y obtenerlo como un dataframe de Pandas:

    Utilizaremos la función read_sql de pandas para obtener los resultados de nuestros querys: 

In [5]:
query = "SHOW TABLES from employee;"
tables = pd.read_sql(query,connection)
tables

,Tables_in_employee
0,departments
1,dept_emp
2,dept_manager
3,employees
4,salaries
5,titles


In [6]:
query = "select * from employee.employees limit 10;"
employees = pd.read_sql(query,connection)

In [7]:
employees

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [8]:
query = "select * from employee.salaries limit 10;"
salaries = pd.read_sql(query,connection)
salaries

,emp_no,salary,from_date,to_date
0,10001,60117,1986-06-26,1987-06-26
1,10001,62102,1987-06-26,1988-06-25
2,10001,66074,1988-06-25,1989-06-25
3,10001,66596,1989-06-25,1990-06-25
4,10001,66961,1990-06-25,1991-06-25
5,10001,71046,1991-06-25,1992-06-24
6,10001,74333,1992-06-24,1993-06-24
7,10001,75286,1993-06-24,1994-06-24
8,10001,75994,1994-06-24,1995-06-24
9,10001,76884,1995-06-24,1996-06-23


In [9]:
query = "select * from employee.titles limit 10;"
titles = pd.read_sql(query,connection)
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
5,10005,Senior Staff,1996-09-12,9999-01-01
6,10005,Staff,1989-09-12,1996-09-12
7,10006,Senior Engineer,1990-08-05,9999-01-01
8,10007,Senior Staff,1996-02-11,9999-01-01
9,10007,Staff,1989-02-10,1996-02-11


In [200]:
query = "select * from employee.departments limit 10;"
departments = pd.read_sql(query,connection)
departments

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing
5,d004,Production
6,d006,Quality Management
7,d008,Research
8,d007,Sales


In [202]:
query = "select * from employee.dept_emp limit 10;"
dept_emp = pd.read_sql(query,connection)
dept_emp

,emp_no,dept_no,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01
5,10006,d005,1990-08-05,9999-01-01
6,10007,d008,1989-02-10,9999-01-01
7,10008,d005,1998-03-11,2000-07-31
8,10009,d006,1985-02-18,9999-01-01
9,10010,d004,1996-11-24,2000-06-26


### 3. Introducción a Joins entre tablas:

#### Teoría de conjuntos: 
    Conjunto: 
        -Una colección de distintos elementos (Tsitsiklis).
        -Una agrupación, colección o reunión de objeto que cumplen una propiedad determinada.
    
    A los objetos del conjunto se denominan elementos. 
    
    
#### Ejemplos:
    1. El conjunto de estudiantes de data wrangling.
    2. El conjunto de letras de "data wrangling". 
    
#### Notación: 
    Los conjuntos se denotan con letras mayúsculas del alfabeto. Los elementos se escriben dentro de llaves.
    
    A = El conjunto de estudiantes de data wrangling. 
    B = El conjunto de letras de "data wrangling".
    
#### Determinación de conjuntos: 
    Extensión: se enumeran o se nombran los elementos del conjunto.
        B = {d,a,t,w,r,n,g,l} 
    Comprensión: Se enuncia la propiedad o la cualidad que distingue a los elementos.
        B = {x/ x es una letra de las palabras "data wrangling"}
    

#### Conjuntos especiales

        Conjunto vacío: aquel en el que ningún elemento cumple con la propiedad conocida como la regla de elegibilidad. 
        (Para cada familia de conjuntos, existe otro conjunto que contiene un elemento de cada uno de aquellos.)

    (Ver axioma de existencia) 

#### Propiedades de los conjuntos 

<img src="set_theory.png" />

- La intersección de conjuntos es conmutativa.
- La intersección de conjuntos es asociativa. 
- La intersección es distributiva respecto a la unión.
- La unión es distributiva respecto a la intersección.
- La unión de un conjunto con el universo es el universo.
- La intersección de un conjunto con su complemento es el conjunto vacío.



#### Joins entre tablas. 

Una de las aplicaciones más palpables de teoría de conjuntos en la ciencia de datos (aparte de teoría de probabilidad) es en manipulación y union entre sets de datos. De cierta manera, podemos pensar en nuestras colecciones de datos como conjuntos de elementos que comparten ciertas características en común. Sin embargo, estas características están dispersas en varias fuentes de información y parte previa al análisis es el proceso de consolidación, por lo cual se vuelve necesario realizar operaciones de conjuntos con ellos. 

Para esto, utilizaremos las keywords JOIN de SQL.


<img src="joins.jpg" />




### Ejemplo 1: A cada empleado, agregarle todos sus títulos y su vigencia.

In [309]:
%%time
### empleados y títulos 
query = """ SELECT
                A.emp_no,
                A.first_name,
                A.last_name,
                B.title,
                B.from_date,
                B.to_date
         FROM employee.employees A  
         LEFT JOIN employee.titles B on A.emp_no = B.emp_no
         WHERE A.emp_no = 11080
"""
emp_titles = pd.read_sql(query,connection)
emp_titles

Wall time: 515 ms


,emp_no,first_name,last_name,title,from_date,to_date
0,11080,Sanjai,Puppe,Engineer,1988-07-04,1993-07-04
1,11080,Sanjai,Puppe,Senior Engineer,1993-07-04,9999-01-01


### Ejercicio 1: 

Obtener el nombre, apellido,salario, titulo y vigencia del mismo para el empleado 12557. 
Su resultado se debe de ver de la siguiente manera:

<img src="ejercicio_1.png" />


In [21]:
query = """
        SELECT
                A.emp_no,
                A.first_name,
                A.last_name,
                C.salary,
                B.title,
                C.from_date,
                C.to_date
         FROM employee.employees A  
         LEFT JOIN employee.titles B on A.emp_no = B.emp_no
         LEFT JOIN employee.salaries C on C.emp_no = A.emp_no
         WHERE A.emp_no = 12557
"""
emp_12557 = pd.read_sql(query,connection)
emp_12557

,emp_no,first_name,last_name,salary,title,from_date,to_date
0,12557,Stafford,Kopetz,41840,Senior Engineer,1988-03-20,1989-03-20
1,12557,Stafford,Kopetz,46021,Senior Engineer,1989-03-20,1990-03-20
2,12557,Stafford,Kopetz,48699,Senior Engineer,1990-03-20,1991-03-20
3,12557,Stafford,Kopetz,51610,Senior Engineer,1991-03-20,1992-03-19
4,12557,Stafford,Kopetz,51351,Senior Engineer,1992-03-19,1993-03-19
5,12557,Stafford,Kopetz,51660,Senior Engineer,1993-03-19,1994-03-19
6,12557,Stafford,Kopetz,52878,Senior Engineer,1994-03-19,1995-03-19
7,12557,Stafford,Kopetz,54053,Senior Engineer,1995-03-19,1996-03-18
8,12557,Stafford,Kopetz,57950,Senior Engineer,1996-03-18,1997-03-18
9,12557,Stafford,Kopetz,58055,Senior Engineer,1997-03-18,1998-03-18


### Subqueries

Explicación Jose. 


### Ejemplo 2: Obtener el salario máximo por empleado.

### Paso 1: Armar un query con el salario máximo por empleado.

In [16]:
%%time
query = """
        SELECT
            emp_no
            ,MAX(salary) max_salary
        FROM employee.salaries
        GROUP BY 1
        LIMIT 100
        """
inner_table =pd.read_sql(query,connection)

Wall time: 325 ms


### Paso 2: Estructura general de un subquery

In [18]:
####

query = """
        SELECT 
        B.first_name
        ,B.last_name, 
        A.* 
        FROM (SELECT
            emp_no
            ,MAX(salary) max_salary
        FROM employee.salaries
        GROUP BY 1
        LIMIT 100) A
        LEFT JOIN employee.employees B on A.emp_no = B.emp_no 
        """
subquery = pd.read_sql(query,connection)
subquery

,first_name,last_name,emp_no,max_salary
0,Georgi,Facello,10001,88958
1,Bezalel,Simmel,10002,72527
2,Parto,Bamford,10003,43699
3,Chirstian,Koblick,10004,74057
4,Kyoichi,Maliniak,10005,94692
...,...,...,...,...
95,Jayson,Mandell,10096,68612
96,Remzi,Waschkowski,10097,70161
97,Sreekrishna,Servieres,10098,56202
98,Valter,Sullins,10099,98538


### Ejercicio 2: Encontrar el último departamento en el que trabajó el empleado con la información del mismo.

In [60]:
%%time
query = """
select D.emp_no, D.first_name, D.last_name, D.dept_no, D.dept_name FROM (
    SELECT 
    A.emp_no, C.first_name, C.last_name, A.dept_no, B.dept_name,
    ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY A.from_date DESC) as ultimo 
    FROM employee.dept_emp A
    JOIN employee.departments B ON A.dept_no = B.dept_no
    JOIN employee.employees C ON C.emp_no = A.emp_no
    GROUP BY 1, 2, 3, 4, 5) D
WHERE D.ultimo = 1;
"""
pd.read_sql(query,connection)


Wall time: 9.21 s


,emp_no,first_name,last_name,dept_no,dept_name
0,10001,Georgi,Facello,d005,Development
1,10002,Bezalel,Simmel,d007,Sales
2,10003,Parto,Bamford,d004,Production
3,10004,Chirstian,Koblick,d004,Production
4,10005,Kyoichi,Maliniak,d003,Human Resources
...,...,...,...,...,...
300019,499995,Dekang,Lichtner,d004,Production
300020,499996,Zito,Baaz,d004,Production
300021,499997,Berhard,Lenart,d005,Development
300022,499998,Patricia,Breugel,d002,Finance


### Partition By

Explicación Jose. 

In [38]:
query = """
    SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,C.dept_name
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
        LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
        GROUP BY 1,2,3,4,5,6
        LIMIT 100
"""
pd.read_sql(query,connection)

,emp_no,birth_date,first_name,last_name,gender,dept_name,seq
0,10001,1953-09-02,Georgi,Facello,M,Development,1
1,10002,1964-06-02,Bezalel,Simmel,F,Sales,1
2,10003,1959-12-03,Parto,Bamford,M,Production,1
3,10004,1954-05-01,Chirstian,Koblick,M,Production,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,Human Resources,1
...,...,...,...,...,...,...,...
95,10088,1954-02-25,Jungsoon,Syrzycki,F,Customer Service,1
96,10088,1954-02-25,Jungsoon,Syrzycki,F,Sales,2
97,10089,1963-03-21,Sudharsan,Flasterstein,F,Sales,1
98,10090,1961-05-30,Kendra,Hofting,M,Development,1


In [62]:
%%time
query = """
    SELECT *
    FROM (
        SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name
            ,A.gender
            ,C.dept_name
            ,ROW_NUMBER() OVER (PARTITION BY A.emp_no ORDER BY B.from_date DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.dept_emp B ON B.emp_no = A.emp_no
        LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
        GROUP BY 1,2,3,4,5,6
    ) A
    WHERE seq = 1
    LIMIT 100
"""
pd.read_sql(query,connection)

Wall time: 4.18 s


,emp_no,birth_date,first_name,last_name,gender,dept_name,seq
0,10001,1953-09-02,Georgi,Facello,M,Development,1
1,10002,1964-06-02,Bezalel,Simmel,F,Sales,1
2,10003,1959-12-03,Parto,Bamford,M,Production,1
3,10004,1954-05-01,Chirstian,Koblick,M,Production,1
4,10005,1955-01-21,Kyoichi,Maliniak,M,Human Resources,1
...,...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,M,Production,1
96,10097,1952-02-27,Remzi,Waschkowski,M,Research,1
97,10098,1961-09-23,Sreekrishna,Servieres,F,Customer Service,1
98,10099,1956-05-25,Valter,Sullins,F,Sales,1


### Ejercicio 3: Encontrar el salario máximo utilizando el Partition By

In [68]:
query = """
    SELECT *
    FROM (
        SELECT
            A.emp_no
            ,A.birth_date
            ,A.first_name
            ,A.last_name,
            B.salary
            ,ROW_NUMBER() OVER (PARTITION BY B.emp_no ORDER BY B.salary DESC) AS seq
        FROM employee.employees A
        LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
        GROUP BY 1,2,3,4, 5
    ) A
    WHERE seq = 1
    LIMIT 100
"""
pd.read_sql(query,connection)

,emp_no,birth_date,first_name,last_name,salary,seq
0,10001,1953-09-02,Georgi,Facello,88958,1
1,10002,1964-06-02,Bezalel,Simmel,72527,1
2,10003,1959-12-03,Parto,Bamford,43699,1
3,10004,1954-05-01,Chirstian,Koblick,74057,1
4,10005,1955-01-21,Kyoichi,Maliniak,94692,1
...,...,...,...,...,...,...
95,10096,1954-09-16,Jayson,Mandell,68612,1
96,10097,1952-02-27,Remzi,Waschkowski,70161,1
97,10098,1961-09-23,Sreekrishna,Servieres,56202,1
98,10099,1956-05-25,Valter,Sullins,98538,1


### CASE WHEN 

In [329]:
%%time
query = """
    SELECT
        YEAR(birth_date)
        ,SUM(CASE WHEN gender = 'M' THEN B.salary END) as Male_Salary
        ,SUM(CASE WHEN gender = 'F' THEN B.salary END) as Female_Salary
    FROM employee.employees A
    LEFT JOIN employee.salaries B ON B.emp_no = A.emp_no
    GROUP BY 1
"""
pd.read_sql(query,connection)


Wall time: 3.24 s


,YEAR(birth_date),Male_Salary,Female_Salary
0,1952,7.724347e+09,5.160503e+09
1,1953,8.362904e+09,5.486954e+09
2,1954,8.338072e+09,5.659551e+09
3,1955,8.502732e+09,5.617556e+09
4,1956,8.208893e+09,5.603599e+09
5,1957,8.295181e+09,5.467040e+09
6,1958,8.447389e+09,5.640520e+09
7,1959,8.544480e+09,5.567662e+09
8,1960,8.258890e+09,5.676864e+09
9,1961,8.323481e+09,5.596837e+09


### Ejercicio 4: Usando la tabla del máximo salario, encontrar la suma de salarios por departamentos

In [106]:
%%time
query = """
    SELECT
        C.dept_name
        ,SUM(CASE WHEN E.gender = 'M' THEN E.max_salary END) as Male_Salary
        ,SUM(CASE WHEN E.gender = 'F' THEN E.max_salary END) as Female_Salary
    FROM (
        SELECT
                A.emp_no
                ,B.gender 
                ,MAX(salary) max_salary
        FROM employee.salaries A
        LEFT JOIN employee.employees B on A.emp_no = B.emp_no
        GROUP BY 1, 2
    ) E
    LEFT JOIN 
    (
        select * FROM (
        SELECT
            B.emp_no,
            B.dept_no
            ,C.dept_name
            ,ROW_NUMBER() OVER (PARTITION BY B.emp_no ORDER BY B.from_date DESC) AS seq
        FROM employee.dept_emp B 
        LEFT JOIN employee.departments C ON C.dept_no = B.dept_no
        GROUP BY 1,2,3) D
        WHERE D.seq = 1
    )
    C ON C.emp_no = E.emp_no
    GROUP BY 1
"""
pd.read_sql(query,connection)

Wall time: 8.04 s


,dept_name,Male_Salary,Female_Salary
0,Customer Service,8.550066e+08,5.697724e+08
1,Development,3.034072e+09,2.016596e+09
2,Finance,7.085662e+08,4.837360e+08
3,Human Resources,5.988062e+08,3.957272e+08
4,Marketing,8.702489e+08,5.682475e+08
5,Production,2.625696e+09,1.759406e+09
6,Quality Management,6.914463e+08,4.686920e+08
7,Research,7.620423e+08,5.071397e+08
8,Sales,2.446449e+09,1.627097e+09


Se usó la primera subconsulta para obtener los salarios máximos y se unió con la subconsulta del departamento actual en el que trabaja cada empleado. Luego se hizo la suma y la discriminación por género.

# Tarea: _Resolución del caso de asesinato_
En esta sección se encuentra la otra parte de la tarea donde se resuelve el asesinato mediante consultas SQL.

### Haciendo la conexión
Para ello se hará la conexión al archivo proveído por los catedráticos.

In [2]:
import sqlite3

In [3]:
con = sqlite3.connect('Lab3/sql-murder-mystery.db')

Se van a ver todas las tablas que hay para resolver el caso.

In [10]:
pd.read_sql("SELECT * FROM sqlite_master;", con)

,type,name,tbl_name,rootpage,sql
0,table,crime_scene_report,crime_scene_report,2,CREATE TABLE crime_scene_report (\n dat...
1,table,drivers_license,drivers_license,3,CREATE TABLE drivers_license (\n id int...
2,table,person,person,4,CREATE TABLE person (\n id integer PRIM...
3,table,facebook_event_checkin,facebook_event_checkin,5,CREATE TABLE facebook_event_checkin (\n ...
4,table,interview,interview,6,CREATE TABLE interview (\n person_id in...
5,table,get_fit_now_member,get_fit_now_member,7,CREATE TABLE get_fit_now_member (\n id ...
6,index,sqlite_autoindex_get_fit_now_member_1,get_fit_now_member,8,None
7,table,get_fit_now_check_in,get_fit_now_check_in,9,CREATE TABLE get_fit_now_check_in (\n m...
8,table,income,income,10,CREATE TABLE income (\n ssn integer PRI...
9,table,solution,solution,11,"CREATE TABLE solution (\n user integer,..."


Según el texto:

`
...el crimen fue un asesinato que ocurrió en algún momento del 15 de enero de 2018...
`

In [11]:
pd.read_sql("SELECT * FROM crime_scene_report;", con)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


Como se puede notar hay tres cosas para filtrar:

* La fecha está en dígitos seguidos, por lo que la fecha de nuestro asesinato sería 20180115.
* Podemos filtrar por ciudad, siendo SQL City nuestra escena del crimen.
* Como el crimen fue un asesinato, usaremos la palabra _murder_.

In [12]:
pd.read_sql("""SELECT * FROM crime_scene_report 
                WHERE type = 'murder'
                AND date = 20180115
                AND city = 'SQL City';""", con)

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


Tenemos un registro, por lo que procedemos la leer la descripción.

In [16]:
desc_crimen = pd.read_sql("""SELECT * FROM crime_scene_report 
                WHERE type = 'murder'
                AND date = 20180115
                AND city = 'SQL City';""", con)
print(desc_crimen['description'][0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


Ya que nos dan información de varias personas, procedemos a ver la tabla de personas para ver si allí podemos conseguir más información al respecto.

In [17]:
pd.read_sql("SELECT * FROM person;", con)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


Ahora filtraremos para encontrar a Annabel (menos mal que no es la terrorífica muñeca que escapó hace unos días del museo Warren) y a el otro testigo.

In [18]:
anabel = pd.read_sql("""SELECT * FROM person
                        WHERE name LIKE '%Annabel%'
                        and address_street_name LIKE '%Franklin Ave%';""", con)

In [19]:
anabel

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [26]:
testigo = pd.read_sql("""SELECT * FROM person
                        WHERE address_street_name LIKE '%Northwestern Dr%'
                        ORDER BY address_number DESC LIMIT 1;""", con)

In [27]:
testigo

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Ya tenemos la información de ambas personas. Ahora toca leer las entrevistas y otras cosas relacionadas con el caso.

In [29]:
anabel_entrevista = pd.read_sql("SELECT * FROM interview WHERE person_id = 16371;", con)
testigo_entrevista = pd.read_sql("SELECT * FROM interview WHERE person_id = 14887;", con)

A ver que dijo la Anabel...

In [31]:
print(anabel_entrevista['transcript'][0])

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


Y a ver que dijo el otro...

In [32]:
print(testigo_entrevista['transcript'][0])

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".


Por lo que dijeron ambos el cuate que mató a la pobre víctima tenía carro con placas H42W, iba al gimnasio y además su bolsa tenía 48Z, ni color que no hay gimnasios en el área porque todos se conocían.

In [34]:
pd.read_sql("SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%' AND gender = 'male';", con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [36]:
pd.read_sql("SELECT * FROM get_fit_now_member WHERE membership_status = 'gold' AND id LIKE '%48Z%';", con)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Voy a ampliar la primera consulta con un JOIN para tener los datos de la persona también, luego veré el check in del GYM.

In [38]:
pd.read_sql("""SELECT * FROM drivers_license
                JOIN person ON person.license_id = drivers_license.id 
                WHERE plate_number LIKE '%H42W%' AND gender = 'male';""", con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,664760,21,71,black,black,male,4H42WR,Nissan,Altima,51739,Tushar Chandra,664760,312,Phi St,137882671
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Todo apunta a **Jeremy Bowers**, veamos el check-in del gimansio y si no que se vaya confesando el chatío.

In [40]:

pd.read_sql("""SELECT * FROM get_fit_now_check_in
            WHERE check_in_date = 20180109 and membership_id = '48Z55';""", con)


,membership_id,check_in_date,check_in_time,check_out_time
0,48Z55,20180109,1530,1700


Ya salió el peine. Todo apunta a que el maje **Jeremy Bowers** fue el que se sonó al tipo.

In [43]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [45]:
pd.read_sql("SELECT * FROM interview WHERE person_id = 67318;", con)['transcript'][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

Sí se lo sonó él, pero fue contratado, es decir hay dos culpables.

In [48]:
pd.read_sql("""SELECT * FROM drivers_license
                JOIN person ON person.license_id = drivers_license.id 
                WHERE car_make LIKE '%Tesla%'
                AND car_model = 'Model S'
                AND hair_color = 'red'
                AND height BETWEEN 65 and 67
                AND gender = 'female';""", con)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id,name,license_id,address_number,address_street_name,ssn
0,918773,48,65,black,red,female,917UU3,Tesla,Model S,78881,Red Korb,918773,107,Camerata Dr,961388910
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S,90700,Regina George,291182,332,Maple Ave,337169072
2,202298,68,66,green,red,female,500123,Tesla,Model S,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [61]:
pd.read_sql("""SELECT * FROM facebook_event_checkin
                WHERE date BETWEEN 20171200 AND 20171299
                AND event_name LIKE '%Symphony Concert%'
                AND person_id IN (78881, 90700, 99716);""", con)

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


Esta última es para verificar si la chava tenía mucho dinero como dijo le cuate.

In [66]:
pd.read_sql("""SELECT * FROM income
                WHERE ssn IN (961388910, 337169072, 987756388);""", con)

,ssn,annual_income
0,961388910,278000
1,987756388,310000


In [67]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')"
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'

In [69]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/eZXg6Uaxd2k" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

c:\users\juan diego\appdata\local\programs\python\python38\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


#### Como música de fondo.